# LOAD DATA

### Dependencias

In [ ]:
# !pip install xarray netCDF4 geopy tqdm skyfield requests pyproj

### Import librerías

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timezone, timedelta
from functools import lru_cache
from geopy.distance import geodesic
from numpy import radians, sin, cos, arctan2, degrees
from pyproj import Transformer
from skyfield.api import load, Topos
from skyfield.almanac import find_discrete, sunrise_sunset
from tqdm.notebook import tqdm
tqdm.pandas()

### Funciones de cálculo
* **Posición, distancia y tiempo solar**

In [ ]:
def adjusted_position_mc(lat, lon, se_x, se_y, n_samples=50):
    lats = np.random.normal(lat, se_y, n_samples)
    lons = np.random.normal(lon, se_x, n_samples)
    return lats.mean(), lons.mean()

# def adjusted_position_mc(lat, lon, se_x, se_y, n_samples=50):
#     # Convertir error estándar (en metros) a grados
#     lat_std_deg = se_y / 111000  # 111 km ≈ 111000 m
#     lon_std_deg = se_x / (111000 * np.cos(np.radians(lat)) + 1e-6)  # evitar división por cero
#     lats = np.random.normal(lat, lat_std_deg, n_samples)
#     lons = np.random.normal(lon, lon_std_deg, n_samples)
#     return lats.mean(), lons.mean()

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arctan2(np.sqrt(a), np.sqrt(1-a))

def calculate_velocity(distance_km, time_sec):
    return (distance_km / time_sec) * 3600 if time_sec>0 else 0

def calculate_acceleration(v1, v2, time_sec):
    return (v2 - v1) / (time_sec/3600) if time_sec>0 else 0

#### **Horas de luz - efemérides**

 Tablas o datos que indican las posiciones de los cuerpos celestes (como el Sol, la Luna, los planetas, etc.) en el cielo, para fechas y horas específicas.

En astronomía, se usan para saber cuándo ocurren eventos importantes, como:

    - Salida y puesta del Sol 🌄🌇

    - Salida y puesta de la Luna 🌕🌘

    - Duración del día (horas de luz)

    - Fases lunares, eclipses, etc.

Por ejemplo, puedes usar _Skyfield_ para:

    - Saber cuándo sale el Sol en una latitud y longitud determinada.

    - Calcular las horas de luz en el Ártico, donde hay fenómenos extremos como el Sol de medianoche o la noche polar.

In [ ]:
eph = load('de421.bsp')
ts = load.timescale()
@lru_cache(maxsize=50000)
def get_daylight(lat, lon, date_str):
    d = datetime.strptime(date_str,'%Y-%m-%d').replace(tzinfo=timezone.utc)
    t0 = ts.utc(d.year, d.month, d.day)
    t1 = ts.utc((d+timedelta(days=1)).year, (d+timedelta(days=1)).month, (d+timedelta(days=1)).day)
    obs = Topos(latitude_degrees=lat, longitude_degrees=lon)
    f = sunrise_sunset(eph, obs)
    times, events = find_discrete(t0, t1, f)
    sunr = suns = None
    for ti, ev in zip(times, events):
        if ev==1: sunr = ti.utc_datetime()
        elif ev==0: suns = ti.utc_datetime()
    if sunr and suns:
        dh = (suns - sunr).total_seconds()/3600
        return dh, False, False
    else:
        alt = eph['Earth']+obs
        sun_alt = alt.at(t0).observe(eph['Sun']).apparent().altaz()[0].degrees
        if sun_alt>0: return 24.0, False, True
        else: return 0.0, True, False

#### **Datos ambientales remotos**

* **Cobertura de hielo**


datos ambientales satelitales en forma de grilla a través del URL que corresponde a un servidor ERDDAP del gobierno de EE.UU., específicamente del NOAA PolarWatch. Proporciona acceso a 

En concreto, este endpoint:

https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4nh1day

se refiere al producto de concentración diaria de hielo marino del hemisferio norte, conocido como:

NSIDC-0051 / G02202 Version 4 — Northern Hemisphere Daily Sea Ice Concentration

    Filtrar por intervalo 2007 -2017

In [ ]:
url = 'https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4nh1day'
ds = xr.open_dataset(url, engine='netcdf4').sel(time=slice('1985-01-01','2017-12-31'))
ds = ds.sel(time=slice('2007-01-01', '2017-12-31'))
vars_env = ['cdr_seaice_conc','temp_surface','wind_speed','cloud_cover']


In [ ]:

@lru_cache(maxsize=50000)
def get_env(ds, var, lat, lon, date_str):
    try:
        return float(ds[var].sel(time=np.datetime64(date_str), latitude=lat, longitude=lon, method='nearest').values)
    except: return np.nan

La proyección polar es una forma de representar zonas cercanas a los polos (como el Ártico) en un mapa plano. En lugar de usar latitud y longitud directamente (como en los mapas comunes), transforma esas coordenadas en un sistema centrado en el polo.

🧊 ¿Por qué se usa?

   Porque en regiones polares, las líneas de longitud se juntan mucho y los mapas se deforman. La proyección polar mantiene mejor las distancias y formas en esas zonas.

📌 En los datos satelitales como los del hielo marino:  

No están en lat/lon directamente, sino en una grilla con una proyección polar.  

Para extraer un valor (como concentración de hielo) en una lat/lon, necesitas convertir esa lat/lon a coordenadas de esa grilla.

In [ ]:

# Definición del transformador: de lat/lon (WGS84) a proyección polar (usamos EPSG:3413 como estándar NOAA Ártico)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3413", always_xy=True)

# Función de conversión
def convert_to_projected(lat, lon):
    x, y = transformer.transform(lon, lat)
    return x, y

### **Datos CSV principal**  

    Cargar y ordenar

In [ ]:
df = pd.read_csv('../data/raw/polarBear_CTCRWlocations_chukchiBeaufort_1985-2017.csv')
df.head()

,UniqueAnimalID,timestamp,mu_lat,mu_lon,se_mu_x,se_mu_y
0,1,7/1/1986 0:00,69.7687,-141.3759,14589,14589
1,1,7/1/1986 6:00,69.7703,-141.3863,12248,12248
2,1,7/1/1986 12:00,69.7718,-141.3942,9375,9375
3,1,7/1/1986 18:00,69.7730,-141.3960,5835,5835
4,1,7/2/1986 0:00,69.7738,-141.3834,2517,2517


    Filtrar por intervalo 2007 - 2017

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df[(df['timestamp'].dt.year >= 2007) & (df['timestamp'].dt.year <= 2017)]
df = df.sort_values(['UniqueAnimalID','timestamp']).reset_index(drop=True)

In [ ]:
batch_size = 10000
res = []

@lru_cache(maxsize=50000)
def get_env(var, x_proj, y_proj, date_str):
    try:
        return float(ds[var].sel(
            time=np.datetime64(date_str),
            xgrid=x_proj,
            ygrid=y_proj,
            method='nearest'
        ).values)
    except:
        return np.nan

In [ ]:

for i in tqdm(range(0, len(df), batch_size), desc='Batches'):
    b = df.iloc[i:i+batch_size].copy()

    # Ajuste de posición con Monte Carlo y rounding
    b['adj_lat'], b['adj_lon'] = zip(*b.progress_apply(
        lambda r: adjusted_position_mc(r['mu_lat'], r['mu_lon'], r['se_mu_x'], r['se_mu_y']), axis=1))
    b['date_str'] = b['timestamp'].dt.strftime('%Y-%m-%d')
    b['lat2'] = b['adj_lat'].round(2)
    b['lon2'] = b['adj_lon'].round(2)
    b['proj_x'], b['proj_y'] = zip(*b.progress_apply(lambda r: convert_to_projected(r['lat2'], r['lon2']), axis=1))


    # Inicializamos columnas vacías para luego asignar
    b['distance_km'] = 0.0
    b['velocity_kmh'] = 0.0
    b['acceleration_kmh2'] = 0.0

    # Procesar por UniqueAnimalID dentro del batch
    for aid, g in b.groupby('UniqueAnimalID'):
        g = g.reset_index()

        if len(g) < 2:
            continue  # saltar grupos con un solo punto

        dists = [0.0]
        vels = [0.0]
        accs = [0.0]

        for j in range(1, len(g)):
            dt = (g.loc[j, 'timestamp'] - g.loc[j - 1, 'timestamp']).total_seconds()
            dk = haversine_distance(g.loc[j - 1, 'adj_lat'], g.loc[j - 1, 'adj_lon'], g.loc[j, 'adj_lat'], g.loc[j, 'adj_lon'])
            dists.append(dk)
            vels.append(calculate_velocity(dk, dt))

        for j in range(1, len(vels)):
            dt_acc = (g.loc[j, 'timestamp'] - g.loc[j - 1, 'timestamp']).total_seconds()
            accs.append(calculate_acceleration(vels[j - 1], vels[j], dt_acc))

        # Ajustar tamaño de accs si es mayor
        if len(accs) > len(g):
            accs = accs[:len(g)]

        # Asignar resultados al DataFrame original 'b' usando el índice original
        b.loc[g['index'], 'distance_km'] = dists
        b.loc[g['index'], 'velocity_kmh'] = vels
        b.loc[g['index'], 'acceleration_kmh2'] = accs

    # Calcular daylight y variables ambientales
    b[['daylight_hours', 'is_polar_night', 'is_midnight_sun']] = b.progress_apply(
        lambda r: pd.Series(get_daylight(r['lat2'], r['lon2'], r['date_str'])), axis=1)

    for var in vars_env:
        b[var] = b.progress_apply(lambda r: get_env(var, r['proj_x'], r['proj_y'], r['date_str']), axis=1)

    res.append(b)

    df_final = pd.concat(res).reset_index(drop=True)


In [ ]:
def bearing_mc(lat1, lon1, lat2, lon2, se_x1, se_y1, se_x2, se_y2, n=30):
    '''
        Función para dirección con propagación de error (Monte Carlo)
    '''
    bearings = []
    for _ in range(n):
        l1 = np.random.normal(lat1, se_y1)
        o1 = np.random.normal(lon1, se_x1)
        l2 = np.random.normal(lat2, se_y2)
        o2 = np.random.normal(lon2, se_x2)

        φ1, φ2 = radians(l1), radians(l2)
        λ1, λ2 = radians(o1), radians(o2)
        y = sin(λ2 - λ1) * cos(φ2)
        x = cos(φ1) * sin(φ2) - sin(φ1) * cos(φ2) * cos(λ2 - λ1)
        θ = degrees(arctan2(y, x))
        bearings.append((θ + 360) % 360)
    return np.mean(bearings)

# Calcular previos para cambio de dirección
df_final['prev_lat'] = df_final.groupby('UniqueAnimalID')['adj_lat'].shift(1)
df_final['prev_lon'] = df_final.groupby('UniqueAnimalID')['adj_lon'].shift(1)
df_final['prev_se_x'] = df_final.groupby('UniqueAnimalID')['se_mu_x'].shift(1)
df_final['prev_se_y'] = df_final.groupby('UniqueAnimalID')['se_mu_y'].shift(1)

# Calcular bearing con error
df_final['bearing'] = df_final.progress_apply(
    lambda r: bearing_mc(
        r['prev_lat'], r['prev_lon'], r['adj_lat'], r['adj_lon'],
        r['prev_se_x'], r['prev_se_y'], r['se_mu_x'], r['se_mu_y']
    ) if pd.notnull(r['prev_lat']) else np.nan,
    axis=1
)

# Calcular cambio de dirección
df_final['prev_bearing'] = df_final.groupby('UniqueAnimalID')['bearing'].shift(1)
df_final['bearing_change'] = df_final.apply(
    lambda r: abs((r['bearing'] - r['prev_bearing'] + 180) % 360 - 180)
    if pd.notnull(r['prev_bearing']) else 0,
    axis=1
)

#### *__Distance_per_day__*

In [ ]:
df_final['date'] = df_final['timestamp'].dt.date
daily_distance = df_final.groupby(['UniqueAnimalID', 'date'])['distance_km'].sum().reset_index()
daily_distance.rename(columns={'distance_km': 'distance_per_day'}, inplace=True)

#### __Merge final__

In [ ]:

# Merge con df_final
df_final = df_final.merge(daily_distance, on=['UniqueAnimalID', 'date'], how='left')


-----------------------------------------------------------------------------
# SAVE RICH DS

In [ ]:
df_final.to_csv('polar_bear_processed_full.csv', index=False)